# Using constraints in optimization loop

Sigurd Carlsen, June 2019.

In real world experiments it is not always possible to maintain the same space as was specified when the experiment was created. For example one could run out of a certain material for the experiment.

In these cases costraints come in handy. Instead of redefining our space we just implement constraints on the parameter values that can be used in the experiment. These constraints can be removed or changed as the exeriment changes.

So far we have 3 types of constraints: Single, Inclusive and Exclusive.

A "Single" constraint enforces a specific dimension to take on a specific value.
For example: x should always equal 2.0.
An "Inclusive" constraint forces values to only be the ones included in the constraint.
For example: x can only take on values between 2.0 and 3.0
An "Exclusive" constraint forces values to only be the ones excluded in the constraint.
For example: x can not take on values between 2.0 and 3.0

This notebook is an example on how to use these constraints.
Note that constraints can only be used in conjunction with ask() and tell()

Let's start by importing our packages

In [1]:
from ProcessOptimizer.learning import GaussianProcessRegressor
from ProcessOptimizer import Optimizer
from ProcessOptimizer.space import constraints
from ProcessOptimizer.space.constraints import Single, Inclusive, Exclusive, Constraints
from ProcessOptimizer.space import Integer, Categorical, Real
import numpy as np
np.random.seed(1234)

%matplotlib inline
import matplotlib.pyplot as plt
plt.set_cmap("viridis")

<Figure size 432x288 with 0 Axes>

Then we define our space

In [2]:
space = [
        Real(1, 4),
        Integer(1, 4),
        Categorical(list('abcd')),
    ]

We initialize our optimizer and asks for 5 samples.

In [3]:
opt = Optimizer(space, "ET")
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.2343118064815393, 1, 'd'], [1.1911959909356105, 2, 'c'], [3.8921320127588457, 2, 'a'], [2.3207350275586087, 4, 'a'], [2.2603252231778823, 4, 'a']]


After the optimizer has been initialized we can add constraints to it.
This is done by first creating a list of all constraints we want to pass on to the optimizer.
Let us start out simple by creating a list with only one constraint.
First we try the "Single" type constraint.

In [4]:
constraints_list = [
    Single(0,2.0,'real')
]

The first argument to "Single" is the index of the dimension for which the constraint should be applied.
The second argument is the value of the constraint, and the third argument is the type of dimension.

Here we have created a constraint on dimension 0. The type of dimension is set to 'real' as the type of dimension 0 in our space object is of type Real. Because dimension is of type 'real' or value 2.0 is of type float

Now we add the constraints to the optimizer and asks for 5 samples to evaluate.
Note that adding constraints should always be done by calling add_constraints() to ensure that the optimizer internally handles new constraints correctly. 

In [5]:
opt.add_constraints(constraints_list)

In [6]:
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.0, 1, 'b'], [2.0, 3, 'd'], [2.0, 1, 'd'], [2.0, 4, 'c'], [2.0, 2, 'd']]


Note how all of the samples satisfies the constraint: Dimension 0 is alawys 2.0

We can remove the constraints again simply by calling remove_constraints()

In [7]:
opt.remove_constraints()
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.9126986889774467, 3, 'c'], [1.9701164177071768, 1, 'a'], [2.6897707948544354, 1, 'b'], [2.1818021011307493, 3, 'b'], [2.1496104600566057, 4, 'b']]


Now let us try with some Inclusive constraints

In [8]:
constraints_list = [
    Inclusive(1,(2,3),'integer'),
    Inclusive(2,('b','c'),'categorical')  
]

Note how we change the dimension_type arguments to 'integer' and 'categorical' to correspond to the "Real" and "Categorical" dimension types of dimension 1 and 2.
The second argument of Inclusive constraints should be a tuple of length 2 for 'integer' and 'real' dimensions. This tuple defines the lower and upper bound of the constraint.
For categorical dimensions the tuple can be more than 2 in length as it describes all the valid values for the dimension.

In [9]:
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[3.261307319755817, 3, 'c'], [3.249128233823785, 3, 'c'], [3.0169548827456554, 2, 'c'], [3.2989259615023494, 3, 'c'], [2.2417848215303335, 2, 'b']]


Now only values between and including 2 and 3 are drawn for dimension 1 and only categorical values of 'b' or 'c' are drawn for dimension 2.
Note that calling add_constraints always overwrites all existing constraints in the optimizer

Exlusive constraints are called the same way as Inclusive constraints but instead of enforcing that values drawn lie inside the bounds of the constraints values are insted forced to be outside of the bounds of the constraints

In [10]:
constraints_list = [
    Exclusive(1,(2,3),'integer'),
    Exclusive(2,('b','c'),'categorical')  
]

In [11]:
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.912688569516774, 1, 'd'], [1.4780143227403975, 4, 'd'], [1.8435681954714562, 4, 'd'], [1.0553334828375456, 4, 'a'], [2.146550819874969, 4, 'd']]


We can use multiple constraints on the same dimension as long as they do not conflict with eachother. For example it is not possible to put two different "Single" constraints on the same dimension.

Here we want values between 1.5 and 2.0 and values between 3.0 and 3.5 to be excluded:

In [18]:
constraints_list = [
    Exclusive(0,(1.5,2.0),'real'),
    Exclusive(0,(3.0,3.5),'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[1.4095737318849202, 4, 'c'], [3.883449567498234, 2, 'c'], [2.1787882813340387, 1, 'b'], [1.0993816336456808, 3, 'a'], [2.332738862880719, 2, 'b'], [2.9670648683764194, 1, 'a'], [1.396904758602636, 4, 'a'], [2.7022622212028287, 3, 'b'], [1.2458561367434886, 4, 'c'], [3.783285716276616, 2, 'b']]


The same can be done with Inclusive constraints

In [22]:
constraints_list = [
    Inclusive(0,(1.0,1.1),'real'),
    Inclusive(0,(2.0,2.1),'real'),
    Inclusive(0,(3.0,3.1),'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[3.0922902576424764, 1, 'c'], [3.0288075703414212, 1, 'c'], [3.0384616060013423, 4, 'd'], [3.0205996792810033, 4, 'a'], [3.0354381854443337, 2, 'a'], [1.0283293777858034, 2, 'd'], [2.015710741999571, 1, 'c'], [1.0120553889598585, 2, 'a'], [3.041019797891082, 1, 'c'], [1.0334408552353442, 4, 'c']]


We can even mix Inclusive and Exclusive constraints for the same dimension

In [23]:
constraints_list = [
    Inclusive(0,(2.0,3.0),'real'),
    Exclusive(0,(2.3,2.7),'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[2.712085887791109, 1, 'd'], [2.227056771517005, 1, 'b'], [2.291557566635598, 2, 'a'], [2.2614188733959946, 2, 'c'], [2.1282214020693466, 4, 'd'], [2.1124389248316535, 3, 'a'], [2.0976753313884697, 4, 'c'], [2.845249687597958, 1, 'a'], [2.1884452386780615, 1, 'c'], [2.29287112090361, 1, 'a']]


Whatch out for creating invalid mixes of constraints. For example: Inclusive(0,(2.0,3.0),'real') and Exclusive(0,(1.5,3.5),'real') leaves no valid values to be drawn